In [ ]:
import os
try:
    import requests
except ImportError:
    os.system('python -m pip install requests')
try:
    import dateutil
except ImportError:
    os.system('python -m pip install python-dateutil')

import requests
import dateutil.parser


def get_enctoken(userid, password, twofa):
    session = requests.Session()
    response = session.post('https://kite.zerodha.com/api/login', data={
        "user_id": userid,
        "password": password
    })
    response = session.post('https://kite.zerodha.com/api/twofa', data={
        "request_id": response.json()['data']['request_id'],
        "twofa_value": twofa,
        "user_id": response.json()['data']['user_id']
    })
    enctoken = response.cookies.get('enctoken')
    if enctoken:
        return enctoken
    else:
        raise Exception("Enter valid details !!!!")


class KiteApp:
    # Products
    PRODUCT_MIS = "MIS"
    PRODUCT_CNC = "CNC"
    PRODUCT_NRML = "NRML"
    PRODUCT_CO = "CO"

    # Order types
    ORDER_TYPE_MARKET = "MARKET"
    ORDER_TYPE_LIMIT = "LIMIT"
    ORDER_TYPE_SLM = "SL-M"
    ORDER_TYPE_SL = "SL"

    # Varities
    VARIETY_REGULAR = "regular"
    VARIETY_CO = "co"
    VARIETY_AMO = "amo"

    # Transaction type
    TRANSACTION_TYPE_BUY = "BUY"
    TRANSACTION_TYPE_SELL = "SELL"

    # Validity
    VALIDITY_DAY = "DAY"
    VALIDITY_IOC = "IOC"

    # Exchanges
    EXCHANGE_NSE = "NSE"
    EXCHANGE_BSE = "BSE"
    EXCHANGE_NFO = "NFO"
    EXCHANGE_CDS = "CDS"
    EXCHANGE_BFO = "BFO"
    EXCHANGE_MCX = "MCX"

    def __init__(self, enctoken):
        self.headers = {"Authorization": f"enctoken {enctoken}"}
        self.session = requests.session()
        self.root_url = "https://api.kite.trade"
        # self.root_url = "https://kite.zerodha.com/oms"
        self.session.get(self.root_url, headers=self.headers)

    def instruments(self, exchange=None):
        data = self.session.get(f"{self.root_url}/instruments",headers=self.headers).text.split("\n")
        Exchange = []
        for i in data[1:-1]:
            row = i.split(",")
            if exchange is None or exchange == row[11]:
                Exchange.append({'instrument_token': int(row[0]), 'exchange_token': row[1], 'tradingsymbol': row[2],
                                 'name': row[3][1:-1], 'last_price': float(row[4]),
                                 'expiry': dateutil.parser.parse(row[5]).date() if row[5] != "" else None,
                                 'strike': float(row[6]), 'tick_size': float(row[7]), 'lot_size': int(row[8]),
                                 'instrument_type': row[9], 'segment': row[10],
                                 'exchange': row[11]})
        return Exchange

    def quote(self, instruments):
        data = self.session.get(f"{self.root_url}/quote", params={"i": instruments}, headers=self.headers).json()["data"]
        return data

    def ltp(self, instruments):
        data = self.session.get(f"{self.root_url}/quote/ltp", params={"i": instruments}, headers=self.headers).json()["data"]
        return data

    def historical_data(self, instrument_token, from_date, to_date, interval, continuous=False, oi=False):
        params = {"from": from_date,
                  "to": to_date,
                  "interval": interval,
                  "continuous": 1 if continuous else 0,
                  "oi": 1 if oi else 0}
        lst = self.session.get(
            f"{self.root_url}/instruments/historical/{instrument_token}/{interval}", params=params,
            headers=self.headers).json()["data"]["candles"]
        records = []
        for i in lst:
            record = {"date": dateutil.parser.parse(i[0]), "open": i[1], "high": i[2], "low": i[3],
                      "close": i[4], "volume": i[5],}
            if len(i) == 7:
                record["oi"] = i[6]
            records.append(record)
        return records

    def margins(self):
        margins = self.session.get(f"{self.root_url}/user/margins", headers=self.headers).json()["data"]
        return margins

    def orders(self):
        orders = self.session.get(f"{self.root_url}/orders", headers=self.headers).json()["data"]
        return orders

    def positions(self):
        positions = self.session.get(f"{self.root_url}/portfolio/positions", headers=self.headers).json()["data"]
        return positions

    def place_order(self, variety, exchange, tradingsymbol, transaction_type, quantity, product, order_type, price=None,
                    validity=None, disclosed_quantity=None, trigger_price=None, squareoff=None, stoploss=None,
                    trailing_stoploss=None, tag=None):
        params = locals()
        del params["self"]
        for k in list(params.keys()):
            if params[k] is None:
                del params[k]
        order_id = self.session.post(f"{self.root_url}/orders/{variety}",
                                     data=params, headers=self.headers).json()["data"]["order_id"]
        return order_id

    def modify_order(self, variety, order_id, parent_order_id=None, quantity=None, price=None, order_type=None,
                     trigger_price=None, validity=None, disclosed_quantity=None):
        params = locals()
        del params["self"]
        for k in list(params.keys()):
            if params[k] is None:
                del params[k]

        order_id = self.session.put(f"{self.root_url}/orders/{variety}/{order_id}",
                                    data=params, headers=self.headers).json()["data"][
            "order_id"]
        return order_id

    def cancel_order(self, variety, order_id, parent_order_id=None):
        order_id = self.session.delete(f"{self.root_url}/orders/{variety}/{order_id}",
                                       data={"parent_order_id": parent_order_id} if parent_order_id else {},
                                       headers=self.headers).json()["data"]["order_id"]
        return order_id


In [ ]:
# get enctoken from kite web 
enctoken =  "******"

kite = KiteApp(enctoken=enctoken)

In [ ]:
kite

In [ ]:
print(kite.margins())
print(kite.orders())
print(kite.positions())


In [ ]:
import datetime
instrument_token = 5633
from_datetime = datetime.datetime.now() - datetime.timedelta(days=20)     # From last & days
to_datetime = datetime.datetime.now()
interval = "minute"
print(kite.historical_data(instrument_token, from_datetime, to_datetime, interval, continuous=False, oi=False))

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import os

In [ ]:
# Import necessary libraries
import pandas as pd
from datetime import datetime, timedelta
import os

# Read the stock symbols from the CSV file
symbol_list = pd.read_csv(r"D:\PythonProject\Trading\Data\nifty.csv", header=0)

# Set the directory path to store data and date range
path_to_store_data = r"D:\PythonProject\Trading\Data\Nifty_500"
from_date = "2016-01-01"
to_date = "2023-10-23"

# Create a directory if it doesn't exist
if not os.path.exists(path_to_store_data + from_date + "to" + to_date):
    os.makedirs(path_to_store_data + from_date + "to" + to_date)
path_to_store_data = path_to_store_data + from_date + "to" + to_date + "/"

# Convert string dates to datetime objects
st = datetime.strptime(from_date, "%Y-%m-%d")
en = datetime.strptime(to_date, "%Y-%m-%d")

# Specify the interval for stock data
interval = "minute"

# Loop through the stock symbols
for i in range(symbol_list.shape[0]):
    # Print current index and stock ID
    print(i)
    ID = symbol_list.iloc[i, 1]
    print(ID)
    symbol_name = symbol_list.iloc[i, 0]
    print(symbol_name)

    # Initialize the start date for data retrieval
    the_date = st
    big_data = pd.DataFrame()

    # Loop until the end date is reached
    while the_date <= en:
        # Define the end date for the data retrieval
        next_date = the_date + timedelta(days=20)
        
        # If the next date exceeds the current date, set it to the end date
        if next_date > datetime.now():
            next_date = en

        # Formulate the date range for data retrieval
        dt_range = f"{the_date}&to={next_date}"
        print(dt_range)

        try:
            # Retrieve historical data for the specified stock and date range
            data = kite.historical_data(ID, the_date, next_date, interval, continuous=False, oi=False)
            df = pd.DataFrame(data)
            
            # Skip if the data length is less than 10
            if len(df) < 10:
                the_date += timedelta(days=50)
                continue  # Skip the rest of the loop if the length is less than 10
            
            # Modify and process the retrieved data
            df["SYMBOL"] = symbol_name
            df.columns = ["TIME", "Open", "High", "Low", "CLOSE", "VOLUME", "SYMBOL"]
            df["TIME"] = df["TIME"].astype(str).replace("+0530", "")
            df["TIME"] = df["TIME"].astype(str).replace("T", " ")
            df["Date"] = pd.to_datetime(df["TIME"]).dt.date
            df["TIME1"] = pd.to_datetime(df["TIME"]).dt.strftime("%H:%M:%S")
            big_data = pd.concat([big_data, df], ignore_index=True)
            print(next_date)
            
            the_date += timedelta(days=20)
        except Exception as e:
            # Handle exceptions if any occur
            print(f"An exception occurred: {e}")

    # Define the file path to save the retrieved data as CSV
    file_path = f"{path_to_store_data}{symbol_name}.csv"
    print(file_path)
    big_data.to_csv(file_path, index=False)
    print(the_date)


In [ ]:
# use multi threading to download ans save data from zerodha web on 1 min time frame 
# time frame can be changed as per requirements 
# data availabe from 2016 only till date 

import os
import pandas as pd
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

def retrieve_data_for_symbol(ID, symbol_name, from_date, to_date, path_to_store_data, interval, kite):
    st = datetime.strptime(from_date, "%Y-%m-%d")
    en = datetime.strptime(to_date, "%Y-%m-%d")
    the_date = st
    big_data = pd.DataFrame()

    while the_date <= en:
        next_date = the_date + timedelta(days=20)
        if next_date > datetime.now():
            next_date = en

        dt_range = f"{the_date}&to={next_date}"
        
        try:
            data = kite.historical_data(ID, the_date, next_date, interval, continuous=False, oi=False)
            df = pd.DataFrame(data)
            
            if len(df) < 10:
                the_date += timedelta(days=50)
                continue  # Skip the rest of the loop if the length is less than 10
            
            df["SYMBOL"] = symbol_name
            df.columns = ["TIME", "Open", "High", "Low", "CLOSE", "VOLUME", "SYMBOL"]
            df["TIME"] = df["TIME"].astype(str).replace("+0530", "")
            df["TIME"] = df["TIME"].astype(str).replace("T", " ")
            df["Date"] = pd.to_datetime(df["TIME"]).dt.date
            df["TIME1"] = pd.to_datetime(df["TIME"]).dt.strftime("%H:%M:%S")
            big_data = pd.concat([big_data, df], ignore_index=True)
            the_date += timedelta(days=20)
        except:
            pass

    file_path = f"{path_to_store_data}{symbol_name}.csv"
    big_data.to_csv(file_path, index=False)

def run_multi_threading(symbol_list, from_date, to_date, path_to_store_data, interval, num_threads, kite):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for i in range(symbol_list.shape[0]):
            ID = symbol_list.iloc[i, 1]
            symbol_name = symbol_list.iloc[i, 0]
            executor.submit(retrieve_data_for_symbol, ID, symbol_name, from_date, to_date, path_to_store_data, interval, kite)

# Main execution
symbol_list = pd.read_csv(r"D:\PythonProject\Trading\Data\nifty.csv", header=0)
path_to_store_data = r"D:\PythonProject\Trading\Data\Nifty_500"
from_date = "2016-01-01"
to_date = "2023-10-23"
interval = "minute"
num_threads = 10  # You can modify the number of threads

if not os.path.exists(f"{path_to_store_data}{from_date}to{to_date}"):
    os.makedirs(f"{path_to_store_data}{from_date}to{to_date}")
path_to_store_data = f"{path_to_store_data}{from_date}to{to_date}/"

# Assuming you have kite initialized for fetching data
# Replace kite with your method of fetching historical data

run_multi_threading(symbol_list, from_date, to_date, path_to_store_data, interval, num_threads, kite)


In [ ]:
print(the_date)
print(next_date)
print(interval)
data = kite.historical_data(4481793, the_date, next_date, interval, continuous=False, oi=False)
data

In [ ]:
data = pd.DataFrame(data)
len(data)